In [1]:
import logging
import os.path
import time
from collections import OrderedDict
import sys

import numpy as np
import torch.nn.functional as F
from torch import optim

from braindecode.models.deep4 import Deep4Net
from braindecode.datasets.bcic_iv_2a import BCICompetition4Set2A
from braindecode.experiments.experiment import Experiment
from braindecode.experiments.monitors import LossMonitor, MisclassMonitor, \
    RuntimeMonitor
from braindecode.experiments.stopcriteria import MaxEpochs, NoDecrease, Or
from braindecode.datautil.iterators import BalancedBatchSizeIterator
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from braindecode.datautil.splitters import split_into_two_sets
from braindecode.torch_ext.constraints import MaxNormDefaultConstraint
from braindecode.torch_ext.util import set_random_seeds, np_to_var
from braindecode.mne_ext.signalproc import mne_apply
from braindecode.datautil.signalproc import (bandpass_cnt,
                                             exponential_running_standardize)
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne
from braindecode.datautil.signal_target import SignalAndTarget

import warnings
warnings.filterwarnings("ignore")

log = logging.getLogger(__name__)
# logging.basicConfig(filename="./outputlog.csv")

In [2]:
data_folder = '/Users/debaojian/OneDrive/OneDrive - UNSW/UNSW/Research/EEG/BCI_Competition/'
subject_id = 1 
low_cut_hz = 4 
ival = [-500, 4000]
max_epochs = 1600
max_increase_epochs = 160
batch_size = 60
high_cut_hz = 38
factor_new = 1e-3
init_block_size = 1000
valid_set_fraction = 0.2
marker_def = OrderedDict([('Left Hand', [1]), ('Right Hand', [2],),
                        ('Foot', [3]), ('Tongue', [4])])   

In [3]:
def load_subject_data(subject_id):
    train_filename = 'A{:02d}T.gdf'.format(subject_id)
    test_filename = 'A{:02d}E.gdf'.format(subject_id)
    train_filepath = os.path.join(data_folder, train_filename)
    test_filepath = os.path.join(data_folder, test_filename)
    train_label_filepath = train_filepath.replace('.gdf', '.mat')
    test_label_filepath = test_filepath.replace('.gdf', '.mat')

    train_loader = BCICompetition4Set2A(
        train_filepath, labels_filename=train_label_filepath)
    test_loader = BCICompetition4Set2A(
        test_filepath, labels_filename=test_label_filepath)
    train_cnt = train_loader.load()
    test_cnt = test_loader.load()
    
    train_cnt = data_preprocessing(train_cnt)
    test_cnt = data_preprocessing(test_cnt)
    
    train_set = create_signal_target_from_raw_mne(train_cnt, marker_def, ival)
    test_set = create_signal_target_from_raw_mne(test_cnt, marker_def, ival)
    
    return SignalAndTarget(np.concatenate((train_set.X, test_set.X), axis=0),
                           np.concatenate((train_set.y, test_set.y), axis=0))

In [4]:
def data_preprocessing(train_cnt):
    train_cnt = train_cnt.drop_channels(['STI 014', 'EOG-left',
                                         'EOG-central', 'EOG-right'])
    assert len(train_cnt.ch_names) == 22
    # lets convert to millvolt for numerical stability of next operations
    train_cnt = mne_apply(lambda a: a * 1e6, train_cnt)
    train_cnt = mne_apply(
        lambda a: bandpass_cnt(a, low_cut_hz, high_cut_hz, train_cnt.info['sfreq'],
                               filt_order=3,
                               axis=1), train_cnt)
    train_cnt = mne_apply(
        lambda a: exponential_running_standardize(a.T, factor_new=factor_new,
                                                  init_block_size=init_block_size,
                                                  eps=1e-4).T,
        train_cnt)
    return train_cnt

In [5]:
def load_train_data(subjects):
    data = load_subject_data(subjects.pop(0))
    x = data.X
    y = data.y
    for s in subjects:
        data = load_subject_data(s)
        x = np.concatenate((x, data.X), axis=0)
        y = np.concatenate((y, data.y), axis=0)
    return SignalAndTarget(x, y)      

In [12]:
def load_single_data(subject_id):
    train_filename = 'A{:02d}T.gdf'.format(subject_id)
    test_filename = 'A{:02d}E.gdf'.format(subject_id)
    train_filepath = os.path.join(data_folder, train_filename)
    test_filepath = os.path.join(data_folder, test_filename)
    train_label_filepath = train_filepath.replace('.gdf', '.mat')
    test_label_filepath = test_filepath.replace('.gdf', '.mat')

    train_loader = BCICompetition4Set2A(
        train_filepath, labels_filename=train_label_filepath)
    test_loader = BCICompetition4Set2A(
        test_filepath, labels_filename=test_label_filepath)
    train_cnt = train_loader.load()
    test_cnt = test_loader.load()
    
    train_cnt = data_preprocessing(train_cnt)
    test_cnt = data_preprocessing(test_cnt)
    
    train_set = create_signal_target_from_raw_mne(train_cnt, marker_def, ival)
    test_set = create_signal_target_from_raw_mne(test_cnt, marker_def, ival)
    
    return train_set, test_set

In [7]:
def run_exp(train_data, test_data, low_cut_hz, model, cuda):
    batch_size = 60
    train_set, valid_set = split_into_two_sets(
        train_data, first_set_fraction=1-valid_set_fraction)

    set_random_seeds(seed=20181221, cuda=cuda)

    n_classes = 4
    n_chans = int(train_set.X.shape[1])
    input_time_length = train_set.X.shape[2]
    if model == 'shallow':
        model = ShallowFBCSPNet(n_chans, n_classes, input_time_length=input_time_length,
                            final_conv_length='auto').create_network()
    elif model == 'deep':
        model = Deep4Net(n_chans, n_classes, input_time_length=input_time_length,
                            final_conv_length='auto').create_network()
    if cuda:
        model.cuda()
    log.info("Model: \n{:s}".format(str(model)))

    optimizer = optim.Adam(model.parameters())

    iterator = BalancedBatchSizeIterator(batch_size=batch_size)

    stop_criterion = Or([MaxEpochs(max_epochs),
                         NoDecrease('valid_misclass', max_increase_epochs)])

    monitors = [LossMonitor(), MisclassMonitor(), RuntimeMonitor()]

    model_constraint = MaxNormDefaultConstraint()

    exp = Experiment(model, train_set, valid_set, test_data, iterator=iterator,
                     loss_function=F.nll_loss, optimizer=optimizer,
                     model_constraint=model_constraint,
                     monitors=monitors,
                     stop_criterion=stop_criterion,
                     remember_best_column='valid_misclass',
                     run_after_early_stop=True, cuda=cuda)
    exp.run()
    return exp

In [17]:
subjects_list = [i for i in range(1, 10)]
subjects_list.remove(9)
train_data = load_train_data(subjects_list)
test_data = load_subject_data(9)

Extracting EDF parameters from /Users/debaojian/OneDrive/OneDrive - UNSW/UNSW/Research/EEG/BCI_Competition/A01T.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 672527  =      0.000 ...  2690.108 secs...
Extracting EDF parameters from /Users/debaojian/OneDrive/OneDrive - UNSW/UNSW/Research/EEG/BCI_Competition/A01E.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 686999  =      0.000 ...  2747.996 secs...
2018-12-22 12:47:01,442 INFO : Trial per class:
Counter({'Tongue': 72, 'Foot': 72, 'Right Hand': 72, 'Left Hand': 72})
2018-12-22 12:47:01,711 INFO : Trial per class:
Counter({'Left Hand': 72, 'Right Hand': 72, 'Foot': 72, 'Tongue': 72})
Extracting EDF parameters from /Users/debaojian/OneDrive/OneDrive - UNSW/UNSW/Research/EEG/BCI_Competition/A02T.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 677168  =      0.000 ...  27

In [14]:
subjects_list = [i for i in range(1, 10)]
subjects_list.remove(9)
train_data, test_data = load_single_data(1)

Extracting EDF parameters from /Users/debaojian/OneDrive/OneDrive - UNSW/UNSW/Research/EEG/BCI_Competition/A01T.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 672527  =      0.000 ...  2690.108 secs...
Extracting EDF parameters from /Users/debaojian/OneDrive/OneDrive - UNSW/UNSW/Research/EEG/BCI_Competition/A01E.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 686999  =      0.000 ...  2747.996 secs...
2018-12-22 12:46:11,878 INFO : Trial per class:
Counter({'Tongue': 72, 'Foot': 72, 'Right Hand': 72, 'Left Hand': 72})
2018-12-22 12:46:12,151 INFO : Trial per class:
Counter({'Left Hand': 72, 'Right Hand': 72, 'Foot': 72, 'Tongue': 72})


In [19]:
train_data.X.shape

(4608, 22, 1125)

In [20]:
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                        level=logging.DEBUG, stream=sys.stdout)
cuda = True
exp = run_exp(train_data, test_data, low_cut_hz, "shallow", cuda)
log.info("Last 10 epochs")
log.info("\n" + str(exp.epochs_df.iloc[-10:]))

2018-12-22 12:49:09,433 INFO : Model: 
Sequential(
  (dimshuffle): Expression(expression=_transpose_time_to_spat)
  (conv_time): Conv2d(1, 40, kernel_size=(25, 1), stride=(1, 1))
  (conv_spat): Conv2d(40, 40, kernel_size=(1, 22), stride=(1, 1), bias=False)
  (bnorm): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_nonlin): Expression(expression=square)
  (pool): AvgPool2d(kernel_size=(75, 1), stride=(15, 1), padding=0)
  (pool_nonlin): Expression(expression=safe_log)
  (drop): Dropout(p=0.5)
  (conv_classifier): Conv2d(40, 4, kernel_size=(69, 1), stride=(1, 1))
  (softmax): LogSoftmax()
  (squeeze): Expression(expression=_squeeze_final_output)
)
2018-12-22 12:49:09,434 INFO : Run until first stop...
2018-12-22 12:49:10,974 INFO : Epoch 0
2018-12-22 12:49:10,974 INFO : train_loss                6.72373
2018-12-22 12:49:10,975 INFO : valid_loss                6.70457
2018-12-22 12:49:10,975 INFO : test_loss                 6.62631
2018-12-22 12:49:

2018-12-22 12:49:51,520 INFO : test_loss                 1.06079
2018-12-22 12:49:51,521 INFO : train_misclass            0.35621
2018-12-22 12:49:51,522 INFO : valid_misclass            0.48048
2018-12-22 12:49:51,522 INFO : test_misclass             0.49479
2018-12-22 12:49:51,523 INFO : runtime                   3.64624
2018-12-22 12:49:51,524 INFO : 
2018-12-22 12:49:53,645 INFO : Time only for training updates: 2.12s
2018-12-22 12:49:55,177 INFO : Epoch 12
2018-12-22 12:49:55,178 INFO : train_loss                0.89174
2018-12-22 12:49:55,179 INFO : valid_loss                0.99764
2018-12-22 12:49:55,179 INFO : test_loss                 0.99648
2018-12-22 12:49:55,180 INFO : train_misclass            0.34780
2018-12-22 12:49:55,180 INFO : valid_misclass            0.42191
2018-12-22 12:49:55,181 INFO : test_misclass             0.42708
2018-12-22 12:49:55,182 INFO : runtime                   3.65174
2018-12-22 12:49:55,182 INFO : 
2018-12-22 12:49:55,188 INFO : New best valid_m

2018-12-22 12:50:39,126 INFO : test_misclass             0.44965
2018-12-22 12:50:39,127 INFO : runtime                   3.65898
2018-12-22 12:50:39,127 INFO : 
2018-12-22 12:50:41,253 INFO : Time only for training updates: 2.12s
2018-12-22 12:50:42,784 INFO : Epoch 25
2018-12-22 12:50:42,785 INFO : train_loss                0.82929
2018-12-22 12:50:42,786 INFO : valid_loss                1.03684
2018-12-22 12:50:42,786 INFO : test_loss                 1.09699
2018-12-22 12:50:42,787 INFO : train_misclass            0.31687
2018-12-22 12:50:42,788 INFO : valid_misclass            0.43384
2018-12-22 12:50:42,788 INFO : test_misclass             0.47743
2018-12-22 12:50:42,789 INFO : runtime                   3.66108
2018-12-22 12:50:42,789 INFO : 
2018-12-22 12:50:44,915 INFO : Time only for training updates: 2.12s
2018-12-22 12:50:46,446 INFO : Epoch 26
2018-12-22 12:50:46,447 INFO : train_loss                0.79419
2018-12-22 12:50:46,447 INFO : valid_loss                0.97531
201

2018-12-22 12:51:26,772 INFO : runtime                   3.66421
2018-12-22 12:51:26,772 INFO : 
2018-12-22 12:51:28,900 INFO : Time only for training updates: 2.13s
2018-12-22 12:51:30,428 INFO : Epoch 38
2018-12-22 12:51:30,428 INFO : train_loss                0.73103
2018-12-22 12:51:30,429 INFO : valid_loss                0.94535
2018-12-22 12:51:30,430 INFO : test_loss                 0.95798
2018-12-22 12:51:30,430 INFO : train_misclass            0.24607
2018-12-22 12:51:30,431 INFO : valid_misclass            0.38503
2018-12-22 12:51:30,432 INFO : test_misclass             0.40104
2018-12-22 12:51:30,433 INFO : runtime                   3.66104
2018-12-22 12:51:30,434 INFO : 
2018-12-22 12:51:32,561 INFO : Time only for training updates: 2.13s
2018-12-22 12:51:34,089 INFO : Epoch 39
2018-12-22 12:51:34,090 INFO : train_loss                0.71646
2018-12-22 12:51:34,090 INFO : valid_loss                0.97741
2018-12-22 12:51:34,091 INFO : test_loss                 0.95004
201

2018-12-22 12:52:18,120 INFO : Epoch 51
2018-12-22 12:52:18,121 INFO : train_loss                0.68320
2018-12-22 12:52:18,122 INFO : valid_loss                0.94463
2018-12-22 12:52:18,122 INFO : test_loss                 0.98347
2018-12-22 12:52:18,123 INFO : train_misclass            0.23820
2018-12-22 12:52:18,123 INFO : valid_misclass            0.38178
2018-12-22 12:52:18,124 INFO : test_misclass             0.39757
2018-12-22 12:52:18,125 INFO : runtime                   3.66215
2018-12-22 12:52:18,125 INFO : 
2018-12-22 12:52:20,256 INFO : Time only for training updates: 2.13s
2018-12-22 12:52:21,785 INFO : Epoch 52
2018-12-22 12:52:21,786 INFO : train_loss                0.71063
2018-12-22 12:52:21,787 INFO : valid_loss                0.87216
2018-12-22 12:52:21,787 INFO : test_loss                 0.96667
2018-12-22 12:52:21,788 INFO : train_misclass            0.25258
2018-12-22 12:52:21,788 INFO : valid_misclass            0.35792
2018-12-22 12:52:21,789 INFO : test_mis

...

2018-12-22 13:03:58,180 INFO : test_misclass             0.39583
2018-12-22 13:03:58,180 INFO : runtime                   3.65663
2018-12-22 13:03:58,181 INFO : 
2018-12-22 13:04:00,312 INFO : Time only for training updates: 2.13s
2018-12-22 13:04:01,837 INFO : Epoch 243
2018-12-22 13:04:01,838 INFO : train_loss                0.61220
2018-12-22 13:04:01,839 INFO : valid_loss                0.91906
2018-12-22 13:04:01,840 INFO : test_loss                 1.01832
2018-12-22 13:04:01,841 INFO : train_misclass            0.17906
2018-12-22 13:04:01,842 INFO : valid_misclass            0.38178
2018-12-22 13:04:01,842 INFO : test_misclass             0.43056
2018-12-22 13:04:01,843 INFO : runtime                   3.65942
2018-12-22 13:04:01,844 INFO : 
2018-12-22 13:04:03,972 INFO : Time only for training updates: 2.13s
2018-12-22 13:04:05,493 INFO : Epoch 244
2018-12-22 13:04:05,494 INFO : train_loss                0.55840
2018-12-22 13:04:05,495 INFO : valid_loss                0.81948
2

2018-12-22 13:04:49,362 INFO : train_misclass            0.15627
2018-12-22 13:04:49,363 INFO : valid_misclass            0.37852
2018-12-22 13:04:49,364 INFO : test_misclass             0.37326
2018-12-22 13:04:49,364 INFO : runtime                   3.65712
2018-12-22 13:04:49,365 INFO : 
2018-12-22 13:04:51,495 INFO : Time only for training updates: 2.13s
2018-12-22 13:04:53,015 INFO : Epoch 257
2018-12-22 13:04:53,016 INFO : train_loss                0.65957
2018-12-22 13:04:53,017 INFO : valid_loss                0.95773
2018-12-22 13:04:53,018 INFO : test_loss                 0.94756
2018-12-22 13:04:53,018 INFO : train_misclass            0.26207
2018-12-22 13:04:53,019 INFO : valid_misclass            0.38612
2018-12-22 13:04:53,020 INFO : test_misclass             0.39757
2018-12-22 13:04:53,020 INFO : runtime                   3.65853
2018-12-22 13:04:53,022 INFO : 
2018-12-22 13:04:55,149 INFO : Time only for training updates: 2.13s
2018-12-22 13:04:56,673 INFO : Epoch 258
2

2018-12-22 13:05:40,554 INFO : valid_loss                0.81337
2018-12-22 13:05:40,554 INFO : test_loss                 0.81784
2018-12-22 13:05:40,555 INFO : train_misclass            0.18584
2018-12-22 13:05:40,555 INFO : valid_misclass            0.32646
2018-12-22 13:05:40,556 INFO : test_misclass             0.31250
2018-12-22 13:05:40,557 INFO : runtime                   3.65969
2018-12-22 13:05:40,558 INFO : 
2018-12-22 13:05:42,686 INFO : Time only for training updates: 2.13s
2018-12-22 13:05:44,208 INFO : Epoch 271
2018-12-22 13:05:44,209 INFO : train_loss                0.52367
2018-12-22 13:05:44,210 INFO : valid_loss                0.85294
2018-12-22 13:05:44,210 INFO : test_loss                 1.05744
2018-12-22 13:05:44,211 INFO : train_misclass            0.13863
2018-12-22 13:05:44,212 INFO : valid_misclass            0.32755
2018-12-22 13:05:44,212 INFO : test_misclass             0.42535
2018-12-22 13:05:44,213 INFO : runtime                   3.65819
2018-12-22 13

2018-12-22 13:06:31,771 INFO : Epoch 284
2018-12-22 13:06:31,772 INFO : train_loss                0.64675
2018-12-22 13:06:31,773 INFO : valid_loss                0.90062
2018-12-22 13:06:31,773 INFO : test_loss                 0.93555
2018-12-22 13:06:31,774 INFO : train_misclass            0.22680
2018-12-22 13:06:31,775 INFO : valid_misclass            0.34056
2018-12-22 13:06:31,775 INFO : test_misclass             0.35938
2018-12-22 13:06:31,776 INFO : runtime                   3.65222
2018-12-22 13:06:31,777 INFO : 
2018-12-22 13:06:33,905 INFO : Time only for training updates: 2.13s
2018-12-22 13:06:35,423 INFO : Epoch 285
2018-12-22 13:06:35,424 INFO : train_loss                0.55589
2018-12-22 13:06:35,425 INFO : valid_loss                0.82612
2018-12-22 13:06:35,426 INFO : test_loss                 0.97174
2018-12-22 13:06:35,426 INFO : train_misclass            0.15464
2018-12-22 13:06:35,427 INFO : valid_misclass            0.33297
2018-12-22 13:06:35,428 INFO : test_m

2018-12-22 13:07:19,322 INFO : 
2018-12-22 13:07:21,453 INFO : Time only for training updates: 2.13s
2018-12-22 13:07:22,981 INFO : Epoch 298
2018-12-22 13:07:22,981 INFO : train_loss                0.52240
2018-12-22 13:07:22,982 INFO : valid_loss                0.95377
2018-12-22 13:07:22,983 INFO : test_loss                 0.94895
2018-12-22 13:07:22,983 INFO : train_misclass            0.13511
2018-12-22 13:07:22,984 INFO : valid_misclass            0.39913
2018-12-22 13:07:22,984 INFO : test_misclass             0.38021
2018-12-22 13:07:22,985 INFO : runtime                   3.65945
2018-12-22 13:07:22,986 INFO : 
2018-12-22 13:07:25,113 INFO : Time only for training updates: 2.13s
2018-12-22 13:07:26,637 INFO : Epoch 299
2018-12-22 13:07:26,638 INFO : train_loss                0.55458
2018-12-22 13:07:26,638 INFO : valid_loss                0.82562
2018-12-22 13:07:26,639 INFO : test_loss                 0.96651
2018-12-22 13:07:26,640 INFO : train_misclass            0.15355
2